In this notebook, we solve the diffusion equation
$$\frac{\delta y}{\delta t} = \frac{\delta^2 y}{\delta x^2} + e^{-Kt} \left( (1 - K) \sin(x) + \frac{(4 - K)}{2} \sin (2 x) + \frac{(9 - K)}{3} \sin (3 x) + \frac{(16 - K)}{4} \sin (4x) + \frac{(64 - K)}{8} \sin (8x) \right) $$

for $x \in [-\pi, \pi]$ and $t \in [0, 1]$ with the initial condition 
$$y(x, 0) = \sin (x) + \frac{\sin (2 x)}{2} + \frac{\sin (3 x)}{3} + \frac{\sin (4x)}{4} + \frac{\sin (8x)}{8}$$

and the Dirichlet boundary condition $y(-\pi, t) = y(\pi, t) = 0$. The reference solution is 
$$y = e^{-Kt} \left( \sin (x) + \frac{\sin (2 x)}{2} + \frac{\sin (3 x)}{3} + \frac{\sin (4x)}{4} + \frac{\sin (8x)}{8} \right)$$

We use $t \in [0, 0.5]$ as the temporal training domain and consider $t \in (0.5, 1]$ as the extrapolation area. $K$ here is a hyperparameter that controls how fast the amplitude of the solution changes over time.

In [ ]:
# import packages

import deepxde as dde
import numpy as np

from deepxde.backend import tf

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  
import random
import pickle

No backend selected.
Finding available backend...


Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.
Instructions for updating:
non-resource variables are not supported in the long term


Found tensorflow.compat.v1
Setting the default backend to "tensorflow.compat.v1". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)
Enable just-in-time compilation with XLA.



In [ ]:
# set up the problem

for i in range(10):

    K = 1 + i/2 

    def pde(x, y):
        dy_t = dde.grad.jacobian(y, x, i=0, j=1)
        dy_xx = dde.grad.hessian(y, x, i=0, j=0)
        d = 1
        return (
            dy_t
            - d * dy_xx
            - tf.exp(-K * x[:, 1:])
            * ((1 - K) * tf.sin(x[:, 0:1])
                + (4 - K) * tf.sin(2 * x[:, 0:1]) / 2
                + (9 - K) * tf.sin(3 * x[:, 0:1]) / 3
                + (16 - K)* tf.sin(4 * x[:, 0:1]) / 4
                + (64 - K) * tf.sin(8 * x[:, 0:1]) / 8
            )
        )

    def func(x):
        return np.exp(- K * x[:, 1:]) * (
            np.sin(x[:, 0:1])
            + np.sin(2 * x[:, 0:1]) / 2
            + np.sin(3 * x[:, 0:1]) / 3
            + np.sin(4 * x[:, 0:1]) / 4
            + np.sin(8 * x[:, 0:1]) / 8
        )


    geom = dde.geometry.Interval(-np.pi, np.pi)
    timedomain = dde.geometry.TimeDomain(0, 0.5)
    geomtime = dde.geometry.GeometryXTime(geom, timedomain)

    bc = dde.icbc.DirichletBC(geomtime, func, lambda _, on_boundary: on_boundary)
    ic = dde.icbc.IC(geomtime, func, lambda _, on_initial: on_initial)
    data = dde.data.TimePDE(
        geomtime,
        pde,
        [bc, ic],
        num_domain= int(K * 320),
        num_boundary= int(K * 20),
        num_initial= int(K * 10),
        solution=func,
        num_test=10000,
    )

    # set up the model
    layer_size = [2] + [50] * 6 + [1]
    activation = "tanh"
    initializer = "Glorot uniform"
    net = dde.nn.FNN(layer_size, activation, initializer)

    model = dde.Model(data, net)

    model.compile("adam", lr=1e-5, metrics=["l2 relative error"])

    def create_data(t):
        x = geom.uniform_points(256, True)
        t = np.full(256, t/100)
        X = np.hstack((x, t.reshape(-1, 1)))
        return X

    print("Training Model with Amplitude Change: ", str(K))

    model.train(iterations=100000)

    # compare the true solution with the predicted solution
    y_pred = np.array([model.predict(create_data(t)) for t in range(100)])
    y_sol = np.array([func(create_data(t)) for t in range(100)])
    
    sol_data = [y_pred, y_sol]

    with open(f'amplitude_change_{K}.pkl', 'wb') as f:
      pickle.dump(sol_data, f)

Compiling model...
Building feed-forward neural network...
'build' took 0.092322 s



/usr/local/lib/python3.10/dist-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


'compile' took 3.105311 s

Training Model with Amplitude Change:  1.0
Training model...

Step      Train loss                        Test loss                         Test metric   
0         [2.50e+01, 3.93e-02, 5.75e-01]    [2.66e+01, 3.93e-02, 5.75e-01]    [8.32e-01]    
1000      [1.74e+01, 6.08e-03, 1.15e-01]    [1.87e+01, 6.08e-03, 1.15e-01]    [2.87e-01]    
2000      [1.72e+01, 5.11e-03, 7.80e-02]    [1.84e+01, 5.11e-03, 7.80e-02]    [2.43e-01]    
3000      [1.68e+01, 2.39e-03, 1.40e-02]    [1.80e+01, 2.39e-03, 1.40e-02]    [1.82e-01]    
4000      [1.66e+01, 4.48e-03, 1.05e-02]    [1.78e+01, 4.48e-03, 1.05e-02]    [2.06e-01]    
5000      [1.57e+01, 4.25e-03, 2.34e-02]    [1.70e+01, 4.25e-03, 2.34e-02]    [2.69e-01]    
6000      [1.14e+01, 2.26e-02, 6.01e-02]    [1.23e+01, 2.26e-02, 6.01e-02]    [4.26e-01]    
7000      [8.77e+00, 6.48e-02, 8.99e-02]    [9.43e+00, 6.48e-02, 8.99e-02]    [4.64e-01]    
8000      [7.33e+00, 2.45e-02, 5.03e-02]    [7.87e+00, 2.45e-02, 5.03e-02]